In [ ]:
import requests
import pandas as pd
import os
import io
import json
import time
import pyodbc           # Metadata uchun
import datetime         # Metadata uchun
import urllib           # Connection string uchun

print("--- Script 1 (Fresh Start): Ingestion, Decoding, Metadata --- START ---")

# --- Konfiguratsiya ---
REPO_RAW_BASE = "https://raw.githubusercontent.com/odilbekmarimov/DemoProject/main/files_final"
COLUMN_MAP_URL = "https://raw.githubusercontent.com/odilbekmarimov/DemoProject/main/column_table_map.json"

file_mapping_config = {
    't01.csv': {'logical_name': 'users', 'table_id': '01'},
    't02.csv': {'logical_name': 'cards', 'table_id': '02'},
    't03.csv': {'logical_name': 'transactions', 'table_id': '03'},
    't04.csv': {'logical_name': 'logs', 'table_id': '04'},
    't05.csv': {'logical_name': 'reports', 'table_id': '05'},
    't07.csv': {'logical_name': 'scheduled_payments', 'table_id': '07'},
}

RAW_DIR = "raw_data"
DECODED_DIR = "decoded_data"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(DECODED_DIR, exist_ok=True)
print(f"Ensured directories exist: '{RAW_DIR}', '{DECODED_DIR}'")

dataframes = {}

DB_SERVER_META = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'
DB_DATABASE_META = 'BankingDB'
USE_TRUSTED_CONNECTION_META = True
driver_meta = '{ODBC Driver 17 for SQL Server}'

create_metadata_table_sql = """
IF OBJECT_ID('dbo.retrieveinfo', 'U') IS NULL -- Jadval mavjudligini tekshirishning ishonchliroq usuli
BEGIN
    CREATE TABLE dbo.retrieveinfo (
        retrieve_id INT IDENTITY(1,1) PRIMARY KEY,
        source_file NVARCHAR(100) NOT NULL,
        retrieved_at DATETIME2 NOT NULL,
        total_rows INT NULL,
        processed_rows INT NULL,
        errors INT DEFAULT 0,
        notes NVARCHAR(MAX) NULL
    );
    PRINT '[Metadata Setup] Table retrieveinfo created successfully.';
END
ELSE
BEGIN
    PRINT '[Metadata Setup] Table retrieveinfo already exists.';
END
"""

# --- Metadata uchun pyodbc Ulanish
cnxn_meta = None
cursor_meta = None
print("\n[Metadata Setup] Setting up connection...")
try:
    if USE_TRUSTED_CONNECTION_META:
        conn_str_meta = (f"DRIVER={driver_meta};SERVER={DB_SERVER_META};"
                         f"DATABASE={DB_DATABASE_META};Trusted_Connection=yes;")
    else:
        pass
    cnxn_meta = pyodbc.connect(conn_str_meta, autocommit=False)
    cursor_meta = cnxn_meta.cursor()
    print("[Metadata Setup] Connection established.")
    # Metadata jadvalini yaratish/tekshirish
    cursor_meta.execute(create_metadata_table_sql)
    cnxn_meta.commit()
    print("[Metadata Setup] Table check/creation completed.")
except Exception as e:
    print(f"[Metadata Setup] ERROR connecting or setting up table: {e}")
    print("[Metadata Setup] Metadata logging will be skipped.")
    cursor_meta = None; cnxn_meta = None

# Column Mapni Yuklash
print(f"\n[Main] Loading column map from: {COLUMN_MAP_URL}")
column_map_data = None
try:
    response_map = requests.get(COLUMN_MAP_URL)
    response_map.raise_for_status()
    column_map_data = response_map.json()
    # JSON tuzilishini tekshirish (agar bo'sh bo'lsa yoki kutulmagan formatda bo'lsa)
    if not isinstance(column_map_data, dict) or not column_map_data:
        raise ValueError("JSON map is empty or not a valid dictionary.")
    print("[Main] JSON mapping successfully loaded and seems valid.")
except Exception as e:
    print(f"[Main] ERROR loading or parsing JSON mapping file: {e}")
    if cnxn_meta: cnxn_meta.close()
    print("--- Script 1 (Fresh Start) --- FAILED ---")
    exit()

#Fayllarni Yuklash, Dekodlash va Metadata Yozish
print("\n[Main] Starting file ingestion and decoding loop...")
for filename, info in file_mapping_config.items():
    file_url = f"{REPO_RAW_BASE}/{filename}"
    logical_name = info['logical_name']
    table_id = info['table_id']

    # Metadata o'zgaruvchilari
    retrieval_start_time = datetime.datetime.now()
    total_rows_read, processed_rows_count = None, None
    error_flag, error_notes = 0, None
    df = pd.DataFrame()

    print(f"\n>>> Processing: {filename} (Logical: {logical_name})")
    try:
        # Faylni yuklash
        print(f"  Downloading from {file_url}...")
        resp_csv = requests.get(file_url)
        resp_csv.raise_for_status()
        print("  Download complete.")

        # CSV ni o'qish
        print("  Reading CSV...")
        try:
            df = pd.read_csv(io.StringIO(resp_csv.text))
        except UnicodeDecodeError:
             print("    Warning: UTF-8 failed. Trying 'latin1'.")
             df = pd.read_csv(io.StringIO(resp_csv.content.decode('latin1')))
        total_rows_read = len(df)
        print(f"  Read complete: {total_rows_read} rows, {len(df.columns)} columns.")

        # Xom faylni saqlash
        raw_path = os.path.join(RAW_DIR, filename)
        print(f"  Saving raw data to {raw_path}...")
        with open(raw_path, 'wb') as f: f.write(resp_csv.content)
        print("  Raw data saved.")

        # Ustunlarni dekodlash
        print("  Decoding columns...")
        table_specific_map = column_map_data.get(table_id, {}).get("columns", {})
        if not table_specific_map:
            print(f"    Warning: No mapping found for table_id '{table_id}'. Keeping original names.")
            decoded_columns = df.columns.tolist()
        else:
            original_columns = df.columns.tolist()
            decoded_columns = [table_specific_map.get(col.split("-")[-1], col) for col in original_columns]
            df.columns = decoded_columns
            # Muvaffaqiyatli dekodlangan nomlarni chiqarish
            changed_cols = {orig: dec for orig, dec in zip(original_columns, decoded_columns) if orig != dec}
            print(f"    Columns decoded/renamed: {len(changed_cols)} (e.g., {list(changed_cols.items())[:3]}...)")
            print(f"    Final columns: {decoded_columns}")
        print("  Decoding complete.")

        processed_rows_count = len(df)

        # Natijani saqlash
        print(f"  Storing DataFrame '{logical_name}' in memory.")
        dataframes[logical_name] = df
        decoded_path = os.path.join(DECODED_DIR, f"decoded_{filename}")
        print(f"  Saving decoded data to {decoded_path}...")
        df.to_csv(decoded_path, index=False, encoding='utf-8')
        print("  Decoded data saved.")

        error_flag = 0
        error_notes = "Success"
        print(f"  >>> Successfully processed {filename}.")

    except requests.exceptions.RequestException as e:
        error_flag = 1; error_notes = f"Download Error: {e}"
        print(f"  ERROR: {error_notes}")
        dataframes[logical_name] = pd.DataFrame()
    except pd.errors.ParserError as e:
        error_flag = 1; error_notes = f"CSV Parsing Error: {e}"
        print(f"  ERROR: {error_notes}")
        dataframes[logical_name] = pd.DataFrame()
    except pd.errors.EmptyDataError as e:
        error_flag = 0; error_notes = f"File is empty: {e}" # Bo'sh fayl xato emas
        print(f"  WARNING: {error_notes}")
        total_rows_read = 0; processed_rows_count = 0
        dataframes[logical_name] = pd.DataFrame()
    except Exception as e:
        error_flag = 1; error_notes = f"Unexpected Error: {e}"
        print(f"  ERROR: {error_notes}")
        dataframes[logical_name] = pd.DataFrame()

    # Metadata yozish
    if cursor_meta and cnxn_meta:
        print(f"  Logging metadata for {filename}...")
        try:
            sql_insert_meta = """
                INSERT INTO retrieveinfo (source_file, retrieved_at, total_rows, processed_rows, errors, notes)
                VALUES (?, ?, ?, ?, ?, ?) """
            meta_data_tuple = (
                filename, retrieval_start_time,
                total_rows_read, processed_rows_count,
                error_flag, str(error_notes)[:4000] if error_notes else None # Limit notes length
            )
            cursor_meta.execute(sql_insert_meta, meta_data_tuple)
            cnxn_meta.commit()
            print("  Metadata logged.")
        except Exception as meta_ex:
            print(f"  ERROR logging metadata: {meta_ex}")
            if cnxn_meta: cnxn_meta.rollback()
    else:
        print("  Skipping metadata logging (connection not available).")

    time.sleep(0.1) # Pauza

# --- Yakunlash ---
print("\n[Main] Ingestion and decoding loop finished.")
print(f"DataFrames in memory: {list(dataframes.keys())}")
print(f"Raw data location: '{RAW_DIR}'")
print(f"Decoded data location: '{DECODED_DIR}'")

if cnxn_meta:
    print("\n[Metadata] Closing connection...")
    if cursor_meta: cursor_meta.close()
    cnxn_meta.close()
    print("[Metadata] Connection closed.")

print("\n--- Script 1 (Fresh Start): Ingestion, Decoding, Metadata --- END ---")


--- Script 1 (Fresh Start): Ingestion, Decoding, Metadata --- START ---
Ensured directories exist: 'raw_data', 'decoded_data'

[Metadata Setup] Setting up connection...
[Metadata Setup] Connection established.
[Metadata Setup] Table check/creation completed.

[Main] Loading column map from: https://raw.githubusercontent.com/odilbekmarimov/DemoProject/main/column_table_map.json
[Main] JSON mapping successfully loaded and seems valid.

[Main] Starting file ingestion and decoding loop...

>>> Processing: t01.csv (Logical: users)
  Download complete.
  Reading CSV...
  Read complete: 600 rows, 8 columns.
  Saving raw data to raw_data\t01.csv...
  Raw data saved.
  Decoding columns...
    Columns decoded/renamed: 8 (e.g., [('01-00', 'id'), ('01-01', 'name'), ('01-02', 'phone_number')]...)
    Final columns: ['id', 'name', 'phone_number', 'email', 'created_at', 'last_active_at', 'is_vip', 'total_balance']
  Decoding complete.
  Storing DataFrame 'users' in memory.
  Saving decoded data to de

In [ ]:
import pandas as pd
import numpy as np
import re
import os

if 'dataframes' not in locals() or not dataframes:
    print("ERROR: 'dataframes' dictionary not found or empty in memory.")
    print("Please run Script 1 (Ingestion) first.")
    # Yoki decoded fayllardan yuklash logikasi
    exit()
else:
     print("Using 'dataframes' dictionary found in memory.")

print("\n--- Script 2 (Fresh Start): Data Cleaning and Transformation --- START ---")

# Tozalash Sozlamalari 
TRANSACTION_AMOUNT_THRESHOLD = 15000
EMAIL_REGEX = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
PHONE_REGEX = r"^(?=.*\d{6,})[\d\s\+\-\(\)]+$"

cleaned_dataframes = {}

def safe_to_datetime(series, **kwargs):
    return pd.to_datetime(series, errors='coerce', **kwargs)
def safe_to_numeric(series, **kwargs):
    return pd.to_numeric(series, errors='coerce', **kwargs)
def safe_to_boolean(series):
    if series.dtype == 'bool': return series
    map_dict = {'true': True, 'false': False, '1': True, '0': False, 1: True, 0: False, 'yes': True, 'no': False}
    # Handle potential non-string values before lower()
    return series.apply(lambda x: str(x).lower() if pd.notna(x) else x).map(map_dict).fillna(False).astype(bool)
def safe_to_int64(series):
     numeric_series = safe_to_numeric(series)
     try: return numeric_series.astype('Int64')
     except: return numeric_series # Agar Int64 bo'lmasa

def clean_dataframe(df_orig, logical_name):
    """ Umumiy tozalash """
    print(f"\n--- Cleaning DataFrame: {logical_name} ---")
    if df_orig is None or df_orig.empty:
        print("DataFrame is empty or None. Skipping.")
        return pd.DataFrame()
    df = df_orig.copy()
    print(f"Original shape: {df.shape}")
    # String ustunlarni tozalash
    for col in df.select_dtypes(include=['object']).columns:
        if col in df.columns and pd.api.types.is_string_dtype(df[col]):
             try: df[col] = df[col].str.strip()
             except: pass
    print(f"Shape after potential stripping: {df.shape}")
    return df


def clean_users(df_users):
    df = clean_dataframe(df_users, 'users')
    if df.empty: return df
    # Ustun nomlari (Script 1 natijasidan)
    id_col, created_at_col, last_active_col, is_vip_col, balance_col, email_col, phone_col = \
        'id', 'created_at', 'last_active_at', 'is_vip', 'total_balance', 'email', 'phone_number'

    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    if created_at_col in df.columns: df[created_at_col] = safe_to_datetime(df[created_at_col])
    if last_active_col in df.columns: df[last_active_col] = safe_to_datetime(df[last_active_col])
    if is_vip_col in df.columns: df[is_vip_col] = safe_to_boolean(df[is_vip_col])
    if balance_col in df.columns: df[balance_col] = safe_to_numeric(df[balance_col]).fillna(0)
    if email_col in df.columns:
        df['is_email_valid'] = df[email_col].apply(lambda x: bool(re.match(EMAIL_REGEX, str(x))) if pd.notna(x) else False)
        print(f"  Email valid count: {df['is_email_valid'].sum()} / {len(df)}")
    if phone_col in df.columns:
        df['is_phone_valid'] = df[phone_col].apply(lambda x: bool(re.match(PHONE_REGEX, str(x))) if pd.notna(x) else False)
        print(f"  Phone valid count: {df['is_phone_valid'].sum()} / {len(df)}")
    # PK bo'yicha dublikatlarni o'chirish
    if id_col in df.columns and not df[id_col].isnull().all():
        df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
        print(f"  Shape after dropping duplicate '{id_col}': {df.shape}")
    return df

def clean_cards(df_cards):
    df = clean_dataframe(df_cards, 'cards')
    if df.empty: return df
    # Ustun nomlari (taxminiy - Skript 1 natijasini tekshiring!)
    id_col, user_id_col, card_num_col, balance_col, limit_col, created_at_col, status_col = \
        'id', 'user_id', 'card_number', 'balance', 'limit_amount', 'created_at', 'status' # Agar status bo'lsa

    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    if user_id_col in df.columns: df[user_id_col] = safe_to_int64(df[user_id_col])
    if balance_col in df.columns: df[balance_col] = safe_to_numeric(df[balance_col]).fillna(0)
    if limit_col in df.columns: df[limit_col] = safe_to_numeric(df[limit_col]).fillna(0)
    if created_at_col in df.columns: df[created_at_col] = safe_to_datetime(df[created_at_col])
    if status_col in df.columns: df[status_col] = df[status_col].astype(str).str.lower().str.strip()
    # Card number odatda string bo'lishi kerak (boshida 0 bo'lishi mumkin)
    if card_num_col in df.columns: df[card_num_col] = df[card_num_col].astype(str)

    required_cols = [col for col in [id_col, user_id_col, card_num_col] if col in df.columns]
    if required_cols: df.dropna(subset=required_cols, inplace=True)
    # PK bo'yicha dublikatlar
    if id_col in df.columns and not df[id_col].isnull().all():
        df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
        print(f"  Shape after dropping duplicate '{id_col}': {df.shape}")
    # Card number bo'yicha dublikatlar (agar u unique bo'lishi kerak bo'lsa)
    if card_num_col in df.columns and not df[card_num_col].isnull().all():
         orig_len = len(df)
         df.drop_duplicates(subset=[card_num_col], keep='first', inplace=True)
         if len(df) < orig_len: print(f"  Shape after dropping duplicate '{card_num_col}': {df.shape}")
    return df

def clean_transactions(df_trans_orig, df_cards_cleaned):
    df = clean_dataframe(df_trans_orig, 'transactions')
    if df.empty: return df
    # Ustun nomlari (taxminiy - Skript 1 natijasini tekshiring!)
    id_col, from_card_col, to_card_col, amount_col, status_col, created_at_col = \
        'id', 'from_card_id', 'to_card_id', 'amount', 'status', 'created_at'
    exceed_trans_thresh_col = 'exceeds_transaction_threshold'
    exceed_card_limit_col = 'exceeds_card_limit'
    card_limit_ref_col = 'card_limit' # Yangi ustun

    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    # Karta ID lari Cards.id ga ishora qiladi deb faraz qilamiz (Int64)
    if from_card_col in df.columns: df[from_card_col] = safe_to_int64(df[from_card_col])
    if to_card_col in df.columns: df[to_card_col] = safe_to_int64(df[to_card_col])
    if amount_col in df.columns: df[amount_col] = safe_to_numeric(df[amount_col]).fillna(0)
    if status_col in df.columns: df[status_col] = df[status_col].astype(str).str.lower().str.strip()
    if created_at_col in df.columns: df[created_at_col] = safe_to_datetime(df[created_at_col])

    required_cols = [col for col in [id_col, from_card_col, amount_col, created_at_col] if col in df.columns]
    if required_cols: df.dropna(subset=required_cols, inplace=True)

    # Flagging
    df[exceed_trans_thresh_col] = False
    if amount_col in df.columns:
         df[exceed_trans_thresh_col] = df[amount_col] > TRANSACTION_AMOUNT_THRESHOLD
         print(f"  Flagged {df[exceed_trans_thresh_col].sum()} transactions > {TRANSACTION_AMOUNT_THRESHOLD}.")

    df[exceed_card_limit_col] = False
    df[card_limit_ref_col] = 0.0
    if df_cards_cleaned is not None and not df_cards_cleaned.empty and from_card_col in df.columns:
        # --- Karta limitini tekshirish (Cards.id bo'yicha) ---
        card_id_in_cards = 'id'         # Cards PK
        limit_col_in_cards = 'limit_amount' # Cards limit ustuni

        if card_id_in_cards in df_cards_cleaned.columns and limit_col_in_cards in df_cards_cleaned.columns:
            print(f"  Merging with cards on: Tx.'{from_card_col}' -> Cards.'{card_id_in_cards}' for limit check.")
            cards_limits = df_cards_cleaned[[card_id_in_cards, limit_col_in_cards]].copy()
            try:
                df_merged = pd.merge(df, cards_limits, left_on=from_card_col, right_on=card_id_in_cards, how='left', suffixes=('', '_card'))
                merged_limit_col = limit_col_in_cards # Agar nomlar bir xil bo'lsa
                if limit_col_in_cards in df.columns: merged_limit_col += '_card' # Agar to'qnashuv bo'lsa

                if merged_limit_col in df_merged.columns:
                     df[card_limit_ref_col] = safe_to_numeric(df_merged[merged_limit_col]).fillna(0)
                     if amount_col in df.columns:
                           df[exceed_card_limit_col] = (df[amount_col] > df[card_limit_ref_col]) & (df[card_limit_ref_col] > 0)
                           print(f"  Flagged {df[exceed_card_limit_col].sum()} transactions exceeding card limit.")
                else: print(f"  Warning: Limit column '{merged_limit_col}' not found after merge.")
            except Exception as e: print(f"  ERROR during merge/limit flag: {e}")
        else: print(f"  Warning: Required columns ('{card_id_in_cards}', '{limit_col_in_cards}') not found in cleaned cards.")
    else: print("  Info: Skipping card limit check (missing data).")

    # PK bo'yicha dublikatlar
    if id_col in df.columns and not df[id_col].isnull().all():
         df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
         print(f"  Shape after dropping duplicate '{id_col}': {df.shape}")
    return df

# Boshqa jadvallar uchun soddalashtirilgan funksiyalar
def clean_logs(df_logs):
    df = clean_dataframe(df_logs, 'logs')
    if df.empty: return df
    id_col, trans_id_col, created_at_col = 'id', 'transaction_id', 'created_at' # Taxminiy nomlar
    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    if trans_id_col in df.columns: df[trans_id_col] = safe_to_int64(df[trans_id_col]) # Tx.id ga ishora qiladi
    if created_at_col in df.columns: df[created_at_col] = safe_to_datetime(df[created_at_col])
    if id_col in df.columns and not df[id_col].isnull().all(): df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
    return df

def clean_reports(df_reports):
    df = clean_dataframe(df_reports, 'reports')
    if df.empty: return df
    id_col, date_col, value_col = 'id', 'report_date', 'metric_value' # Taxminiy nomlar
    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    if date_col in df.columns: df[date_col] = safe_to_datetime(df[date_col])
    if value_col in df.columns: df[value_col] = safe_to_numeric(df[value_col]).fillna(0)
    if id_col in df.columns and not df[id_col].isnull().all(): df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
    return df

def clean_scheduled_payments(df_sched):
    df = clean_dataframe(df_sched, 'scheduled_payments')
    if df.empty: return df
    id_col, from_card_col, amount_col, date_col, status_col = \
        'id', 'from_card_id', 'amount', 'scheduled_date', 'status' # Taxminiy nomlar
    if id_col in df.columns: df[id_col] = safe_to_int64(df[id_col])
    if from_card_col in df.columns: df[from_card_col] = safe_to_int64(df[from_card_col]) # Cards.id ga ishora qiladi
    if amount_col in df.columns: df[amount_col] = safe_to_numeric(df[amount_col]).fillna(0)
    if date_col in df.columns: df[date_col] = safe_to_datetime(df[date_col])
    if status_col in df.columns: df[status_col] = df[status_col].astype(str).str.lower().str.strip()
    required_cols = [col for col in [id_col, from_card_col, amount_col, date_col] if col in df.columns]
    if required_cols: df.dropna(subset=required_cols, inplace=True)
    if id_col in df.columns and not df[id_col].isnull().all(): df.drop_duplicates(subset=[id_col], keep='first', inplace=True)
    return df

#Asosiy Tozalash Jarayonini Bajarish
print("\n[Main] Starting cleaning process for all DataFrames...")

# Har bir DataFrame uchun tozalash funksiyasini chaqirish
# .get() xavfsizlik uchun ishlatiladi (agar DF yuklanmagan bo'lsa)
cleaned_dataframes['users'] = clean_users(dataframes.get('users'))
cleaned_dataframes['cards'] = clean_cards(dataframes.get('cards'))
# Tranzaksiyalarni tozalash uchun TOZALANGAN kartalar kerak
cleaned_dataframes['transactions'] = clean_transactions(
                                        dataframes.get('transactions'),
                                        cleaned_dataframes.get('cards')
                                     )
cleaned_dataframes['logs'] = clean_logs(dataframes.get('logs'))
cleaned_dataframes['reports'] = clean_reports(dataframes.get('reports'))
cleaned_dataframes['scheduled_payments'] = clean_scheduled_payments(dataframes.get('scheduled_payments'))

print("\n[Main] Data cleaning and transformation process finished.")
# Tozalangan DFlar haqida qisqacha ma'lumot
for name, df_clean in cleaned_dataframes.items():
    if df_clean is not None and not df_clean.empty:
        print(f"  Cleaned '{name}': {df_clean.shape}, Columns: {df_clean.columns.tolist()}")
    else:
        print(f"  Cleaned '{name}': Empty or None")

# --- Ixtiyoriy: Tozalangan ma'lumotlarni faylga saqlash ---
CLEANED_DIR = "cleaned_data"
os.makedirs(CLEANED_DIR, exist_ok=True)
print(f"\n[Main] Saving cleaned data to '{CLEANED_DIR}' directory...")
for name, df in cleaned_dataframes.items():
    if df is not None and not df.empty:
        try:
            save_path = os.path.join(CLEANED_DIR, f"cleaned_{name}.csv")
            # Sana formatini saqlashda muammo bo'lmasligi uchun
            df.to_csv(save_path, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S.%f')
            print(f"  Saved: {save_path}")
        except Exception as e:
            print(f"  Error saving cleaned DataFrame '{name}': {e}")
    else:
        print(f"  Skipping saving for '{name}'.")

print("\n--- Script 2 (Fresh Start): Data Cleaning and Transformation --- END ---")

Using 'dataframes' dictionary found in memory.

--- Script 2 (Fresh Start): Data Cleaning and Transformation --- START ---

[Main] Starting cleaning process for all DataFrames...

--- Cleaning DataFrame: users ---
Original shape: (600, 8)
Shape after potential stripping: (600, 8)
  Email valid count: 600 / 600
  Phone valid count: 600 / 600
  Shape after dropping duplicate 'id': (600, 10)

--- Cleaning DataFrame: cards ---
Original shape: (1200, 7)
Shape after potential stripping: (1200, 7)
  Shape after dropping duplicate 'id': (1200, 7)

--- Cleaning DataFrame: transactions ---
Original shape: (3000, 7)
Shape after potential stripping: (3000, 7)
  Flagged 2998 transactions > 15000.
  Merging with cards on: Tx.'from_card_id' -> Cards.'id' for limit check.
  Flagged 0 transactions exceeding card limit.
  Shape after dropping duplicate 'id': (3000, 10)

--- Cleaning DataFrame: logs ---
Original shape: (2014, 4)
Shape after potential stripping: (2014, 4)

--- Cleaning DataFrame: reports 

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import os
import urllib
import datetime # Datetime konvertatsiyasi uchun

print("--- TEST SCRIPT (NumPy Fix): Load ONLY Users Table --- START ---")

if 'cleaned_dataframes' not in locals() or not cleaned_dataframes.get('users') is not None and cleaned_dataframes.get('users').empty:
    print("ERROR: 'cleaned_dataframes['users']' not found or empty. Run Script 1 & 2 first.")
    exit()
else:
    print("Using 'cleaned_dataframes['users']'. Shape:", cleaned_dataframes['users'].shape)

DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'
DB_DATABASE = 'BankingDB'
USE_TRUSTED_CONNECTION = True
driver = '{ODBC Driver 17 for SQL Server}'

cnxn = None
cursor = None
print("\n[Test Load] Setting up connection (AUTOCOMMIT=FALSE)...")
try:
    conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};"
                 f"DATABASE={DB_DATABASE};Trusted_Connection=yes;")
    cnxn = pyodbc.connect(conn_str, autocommit=False)
    cursor = cnxn.cursor()
    print("[Test Load] Connection established.")
except Exception as e:
    print(f"[Test Load] ERROR connecting: {e}")
    exit()

 SQL Type Function
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype): return 'BIGINT'
    elif pd.api.types.is_float_dtype(dtype): return 'FLOAT'
    elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME2'
    elif pd.api.types.is_bool_dtype(dtype): return 'BIT'
    else: return 'NVARCHAR(MAX)'

logical_name = 'users'
final_table_name = 'Users'
df_cleaned = cleaned_dataframes[logical_name]
pk_col = 'id'

try:
    print(f"\n[Test Load] STEP 0: Dropping table '{final_table_name}' if exists...")
    try:
        cursor.execute(f"DROP TABLE IF EXISTS [{final_table_name}]")
        cnxn.commit()
        print(f"  Table '{final_table_name}' dropped. Commit successful.")
    except Exception as e:
        print(f"  ERROR dropping table '{final_table_name}': {e}")
        raise

    print(f"\n[Test Load] STEP 1a: Creating table '{final_table_name}'...")
    column_defs = []
    valid_sql_cols = []
    for col_name in df_cleaned.columns:
        if not isinstance(col_name, str) or not col_name: continue
        sql_col_name = col_name
        sql_type = get_sql_type(df_cleaned[col_name].dtype)
        column_defs.append(f"[{sql_col_name}] {sql_type}")
        valid_sql_cols.append(sql_col_name)
    pk_def = f", CONSTRAINT PK_{final_table_name} PRIMARY KEY ([{pk_col}])" if pk_col in valid_sql_cols else ""
    create_sql = f"CREATE TABLE [{final_table_name}] ({', '.join(column_defs)} {pk_def} )"
    cursor.execute(create_sql)
    cnxn.commit()
    print(f"  Table '{final_table_name}' created. Commit successful.")

    print(f"\n[Test Load] STEP 1b: Preparing and inserting {len(df_cleaned)} rows...")
    sql_cols_str = ', '.join([f'[{c}]' for c in valid_sql_cols])
    placeholders = ', '.join('?' * len(valid_sql_cols))
    sql_insert = f"INSERT INTO [{final_table_name}] ({sql_cols_str}) VALUES ({placeholders})"

    data_tuples = []
    for row in df_cleaned[valid_sql_cols].itertuples(index=False, name=None):
        processed_row = []
        for item in row:
            if pd.isna(item): # Check for NaN, NaT, pd.NA
                processed_row.append(None)
            elif isinstance(item, (np.int64, np.int32, np.int16, np.int8)):
                processed_row.append(int(item)) # NumPy int -> Python int
            elif isinstance(item, (np.float64, np.float32, np.float16)):
                processed_row.append(float(item)) # NumPy float -> Python float
            elif isinstance(item, (np.bool_)):
                 processed_row.append(bool(item)) # NumPy bool -> Python bool

            else:
                processed_row.append(item) # Qolgan turlarni o'zgartirmaymiz
        data_tuples.append(tuple(processed_row))
    # --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

    if not data_tuples:
        print("    Warning: No data tuples to insert.")
    else:
        print(f"  Executing INSERT statement for {len(data_tuples)} rows...")
        cursor.fast_executemany = True
        cursor.executemany(sql_insert, data_tuples)
        inserted_rows = cursor.rowcount
        cnxn.commit() # <<< COMMIT after successful INSERT
        print(f"  SUCCESS: Inserted {inserted_rows} rows. Commit successful.")

    print(f"\n[Test Load] Successfully processed '{final_table_name}'.")

except pyodbc.Error as db_e: # Faqat pyodbc xatolarini ushlash
    print(f"\n[Test Load] A DATABASE error occurred: {db_e}")
    print(f"  SQL State: {db_e.args[0]}")
    print("Attempting to rollback...")
    if cnxn: cnxn.rollback()
    print("Rollback attempted.")
except Exception as e: # Boshqa Python xatolari
    print(f"\n[Test Load] An unexpected PYTHON error occurred: {e}")
    print("Attempting to rollback...")
    if cnxn: cnxn.rollback()
    print("Rollback attempted.")
finally:
    print("\n[Test Load] Closing connection...")
    if cursor: cursor.close()
    if cnxn: cnxn.close()
    print("[Test Load] Connection closed.")

print("\n--- TEST SCRIPT (NumPy Fix): Load ONLY Users Table --- END ---")

--- TEST SCRIPT (NumPy Fix): Load ONLY Users Table --- START ---
Using 'cleaned_dataframes['users']'. Shape: (600, 10)

[Test Load] Setting up connection (AUTOCOMMIT=FALSE)...
[Test Load] Connection established.

[Test Load] STEP 0: Dropping table 'Users' if exists...
  Table 'Users' dropped. Commit successful.

[Test Load] STEP 1a: Creating table 'Users'...
  Table 'Users' created. Commit successful.

[Test Load] STEP 1b: Preparing and inserting 600 rows...
  Executing INSERT statement for 600 rows...
  SUCCESS: Inserted -1 rows. Commit successful.

[Test Load] Successfully processed 'Users'.

[Test Load] Closing connection...
[Test Load] Connection closed.

--- TEST SCRIPT (NumPy Fix): Load ONLY Users Table --- END ---


In [ ]:
import pandas as pd
import pyodbc
import numpy as np # NumPy import qilingan
import urllib
import datetime

print("\n--- Step 3C (NumPy 2.0 Fix): Load Table 'Cards' --- START ---")
logical_name = 'cards'; final_table_name = 'Cards'; pk_col = 'id'
card_number_col = 'card_number'

if 'cleaned_dataframes' not in locals() or cleaned_dataframes.get(logical_name) is None or cleaned_dataframes.get(logical_name).empty:
    print(f"ERROR/SKIP: cleaned_dataframes['{logical_name}'] not found or empty.")
else:
    df_cleaned = cleaned_dataframes[logical_name]
    print(f"Processing '{final_table_name}' with {len(df_cleaned)} rows.")
    DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'; DB_DATABASE = 'BankingDB'; USE_TRUSTED_CONNECTION = True; driver = '{ODBC Driver 17 for SQL Server}'
    cnxn = None; cursor = None

    # --- SQL Type Function ---
    def get_sql_type(column_name, dtype):
        if column_name == card_number_col: return 'NVARCHAR(50)'
        elif pd.api.types.is_integer_dtype(dtype): return 'BIGINT'
        elif pd.api.types.is_float_dtype(dtype): return 'FLOAT'
        elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME2'
        elif pd.api.types.is_bool_dtype(dtype): return 'BIT'
        else: return 'NVARCHAR(MAX)'

    def convert_value(item):
        if pd.isna(item): return None
        # Kengroq tekshiruv NumPy 2.0+ uchun
        if isinstance(item, (np.integer)): # Barcha NumPy integer turlari uchun
            return int(item)
        if isinstance(item, (np.floating)): # Barcha NumPy float turlari uchun
            return float(item)
        if isinstance(item, (np.bool_)): # NumPy boolean uchun (pastki chiziq bilan)
            return bool(item)
        # Pandas Timestamp ni ham o'girish (agar kerak bo'lsa)
        # if isinstance(item, pd.Timestamp): return item.to_pydatetime()
        return item # Qolganlarini o'zgartirmaymiz

    try:
        conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};DATABASE={DB_DATABASE};Trusted_Connection=yes;")
        cnxn = pyodbc.connect(conn_str, autocommit=False)
        cursor = cnxn.cursor()
        print("  Connection established.")

        # CREATE TABLE
        print(f"  Creating table '{final_table_name}'...")
        column_defs, valid_sql_cols = [], []
        for col_name in df_cleaned.columns:
            if not isinstance(col_name, str) or not col_name: continue
            sql_col_name=col_name
            sql_type=get_sql_type(sql_col_name, df_cleaned[col_name].dtype)
            column_defs.append(f"[{sql_col_name}] {sql_type}"); valid_sql_cols.append(sql_col_name)
        pk_def = f", CONSTRAINT PK_{final_table_name} PRIMARY KEY ([{pk_col}])" if pk_col in valid_sql_cols else ""
        unique_card_def = f", CONSTRAINT UQ_{final_table_name}_CardNumber UNIQUE ([{card_number_col}])" if card_number_col in valid_sql_cols else ""
        create_sql = f"CREATE TABLE [{final_table_name}] ({', '.join(column_defs)} {pk_def} {unique_card_def} )"
        cursor.execute(f"DROP TABLE IF EXISTS [{final_table_name}]")
        cursor.execute(create_sql); print(f"  Table '{final_table_name}' created.")

        # INSERT INTO (YANGILANGAN convert_value bilan)
        print(f"  Preparing and inserting rows...")
        sql_cols_str = ', '.join([f'[{c}]' for c in valid_sql_cols])
        placeholders = ', '.join('?' * len(valid_sql_cols))
        sql_insert = f"INSERT INTO [{final_table_name}] ({sql_cols_str}) VALUES ({placeholders})"
        # >>> data_tuples endi yangi convert_value ni ishlatadi <<<
        data_tuples = [tuple(convert_value(item) for item in row) for row in df_cleaned[valid_sql_cols].itertuples(index=False, name=None)]
        if data_tuples:
            cursor.fast_executemany = True; cursor.executemany(sql_insert, data_tuples)
            print(f"  SUCCESS: Inserted {cursor.rowcount} rows.")
        else: print("    Warning: No data to insert.")
        cnxn.commit(); print(f"  Changes for '{final_table_name}' committed.")

    except Exception as e: print(f"  ERROR processing table '{final_table_name}': {e}"); cnxn.rollback()
    finally:
        if cursor: cursor.close()
        if cnxn: cnxn.close()
        print("  Connection closed.")
print("--- Step 3C (NumPy 2.0 Fix): Load Table 'Cards' --- END ---")


--- Step 3C (NumPy 2.0 Fix): Load Table 'Cards' --- START ---
Processing 'Cards' with 1200 rows.
  Connection established.
  Creating table 'Cards'...
  Table 'Cards' created.
  Preparing and inserting rows...
  SUCCESS: Inserted -1 rows.
  Changes for 'Cards' committed.
  Connection closed.
--- Step 3C (NumPy 2.0 Fix): Load Table 'Cards' --- END ---


In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import urllib
import datetime

print("\n--- Step 3D: Load Table 'Transactions' --- START ---")
logical_name = 'transactions'; final_table_name = 'Transactions'; pk_col = 'id'

if 'cleaned_dataframes' not in locals() or cleaned_dataframes.get(logical_name) is None or cleaned_dataframes.get(logical_name).empty:
    print(f"ERROR/SKIP: cleaned_dataframes['{logical_name}'] not found or empty.")
else:
    df_cleaned = cleaned_dataframes[logical_name]
    print(f"Processing '{final_table_name}' with {len(df_cleaned)} rows.")
    DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'; DB_DATABASE = 'BankingDB'; USE_TRUSTED_CONNECTION = True; driver = '{ODBC Driver 17 for SQL Server}'
    cnxn = None; cursor = None
    # --- SQL Type Function ---
    def get_sql_type(dtype): # Column name not needed here unless specific override
        if pd.api.types.is_integer_dtype(dtype): return 'BIGINT'
        elif pd.api.types.is_float_dtype(dtype): return 'FLOAT'
        elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME2'
        elif pd.api.types.is_bool_dtype(dtype): return 'BIT'
        else: return 'NVARCHAR(MAX)'
    # --- NumPy/Pandas ni Python ga O'girish Funksiyasi ---
    def convert_value(item):
        if pd.isna(item): return None
        if isinstance(item, (np.integer)): return int(item)
        if isinstance(item, (np.floating)): return float(item)
        if isinstance(item, (np.bool_)): return bool(item)
        return item
    try:
        conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};DATABASE={DB_DATABASE};Trusted_Connection=yes;")
        cnxn = pyodbc.connect(conn_str, autocommit=False)
        cursor = cnxn.cursor()
        print("  Connection established.")
        # CREATE TABLE
        print(f"  Creating table '{final_table_name}'...")
        column_defs, valid_sql_cols = [], []
        for col_name in df_cleaned.columns:
            if not isinstance(col_name, str) or not col_name: continue
            sql_col_name=col_name; sql_type=get_sql_type(df_cleaned[col_name].dtype)
            column_defs.append(f"[{sql_col_name}] {sql_type}"); valid_sql_cols.append(sql_col_name)
        pk_def = f", CONSTRAINT PK_{final_table_name} PRIMARY KEY ([{pk_col}])" if pk_col in valid_sql_cols else ""
        create_sql = f"CREATE TABLE [{final_table_name}] ({', '.join(column_defs)} {pk_def} )" # Removed UNIQUE constraint
        cursor.execute(f"DROP TABLE IF EXISTS [{final_table_name}]")
        cursor.execute(create_sql); print(f"  Table '{final_table_name}' created.")
        # INSERT INTO
        print(f"  Preparing and inserting rows...")
        sql_cols_str = ', '.join([f'[{c}]' for c in valid_sql_cols])
        placeholders = ', '.join('?' * len(valid_sql_cols))
        sql_insert = f"INSERT INTO [{final_table_name}] ({sql_cols_str}) VALUES ({placeholders})"
        data_tuples = [tuple(convert_value(item) for item in row) for row in df_cleaned[valid_sql_cols].itertuples(index=False, name=None)]
        if data_tuples:
            cursor.fast_executemany = True; cursor.executemany(sql_insert, data_tuples)
            print(f"  SUCCESS: Inserted {cursor.rowcount} rows.")
        else: print("    Warning: No data to insert.")
        cnxn.commit(); print(f"  Changes for '{final_table_name}' committed.")
    except Exception as e: print(f"  ERROR processing table '{final_table_name}': {e}"); cnxn.rollback()
    finally:
        if cursor: cursor.close()
        if cnxn: cnxn.close()
        print("  Connection closed.")
print(f"--- Step 3D: Load Table '{final_table_name}' --- END ---")


--- Step 3D: Load Table 'Transactions' --- START ---
Processing 'Transactions' with 3000 rows.
  Connection established.
  Creating table 'Transactions'...
  Table 'Transactions' created.
  Preparing and inserting rows...
  SUCCESS: Inserted -1 rows.
  Changes for 'Transactions' committed.
  Connection closed.
--- Step 3D: Load Table 'Transactions' --- END ---


In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import urllib
import datetime

print("\n--- Step 3E: Load Table 'Logs' --- START ---")
logical_name = 'logs'; final_table_name = 'Logs'; pk_col = 'id' # <<< Moslashtiring

if 'cleaned_dataframes' not in locals() or cleaned_dataframes.get(logical_name) is None or cleaned_dataframes.get(logical_name).empty:
    print(f"ERROR/SKIP: cleaned_dataframes['{logical_name}'] not found or empty.")
else:
    df_cleaned = cleaned_dataframes[logical_name]
    print(f"Processing '{final_table_name}' with {len(df_cleaned)} rows.")
    DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'; DB_DATABASE = 'BankingDB'; USE_TRUSTED_CONNECTION = True; driver = '{ODBC Driver 17 for SQL Server}'
    cnxn = None; cursor = None
    def get_sql_type(dtype):
        if pd.api.types.is_integer_dtype(dtype): return 'BIGINT'
        elif pd.api.types.is_float_dtype(dtype): return 'FLOAT'
        elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME2'
        elif pd.api.types.is_bool_dtype(dtype): return 'BIT'
        else: return 'NVARCHAR(MAX)'
    def convert_value(item):
        if pd.isna(item): return None
        if isinstance(item, (np.integer)): return int(item)
        if isinstance(item, (np.floating)): return float(item)
        if isinstance(item, (np.bool_)): return bool(item)
        return item
    try:
        conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};DATABASE={DB_DATABASE};Trusted_Connection=yes;")
        cnxn = pyodbc.connect(conn_str, autocommit=False)
        cursor = cnxn.cursor()
        print("  Connection established.")
        # CREATE TABLE
        print(f"  Creating table '{final_table_name}'...")
        column_defs, valid_sql_cols = [], []
        for col_name in df_cleaned.columns:
            if not isinstance(col_name, str) or not col_name: continue
            sql_col_name=col_name; sql_type=get_sql_type(df_cleaned[col_name].dtype)
            column_defs.append(f"[{sql_col_name}] {sql_type}"); valid_sql_cols.append(sql_col_name)
        pk_def = f", CONSTRAINT PK_{final_table_name} PRIMARY KEY ([{pk_col}])" if pk_col in valid_sql_cols else ""
        create_sql = f"CREATE TABLE [{final_table_name}] ({', '.join(column_defs)} {pk_def} )"
        cursor.execute(f"DROP TABLE IF EXISTS [{final_table_name}]")
        cursor.execute(create_sql); print(f"  Table '{final_table_name}' created.")
        # INSERT INTO
        print(f"  Preparing and inserting rows...")
        sql_cols_str = ', '.join([f'[{c}]' for c in valid_sql_cols])
        placeholders = ', '.join('?' * len(valid_sql_cols))
        sql_insert = f"INSERT INTO [{final_table_name}] ({sql_cols_str}) VALUES ({placeholders})"
        data_tuples = [tuple(convert_value(item) for item in row) for row in df_cleaned[valid_sql_cols].itertuples(index=False, name=None)]
        if data_tuples:
            cursor.fast_executemany = True; cursor.executemany(sql_insert, data_tuples)
            print(f"  SUCCESS: Inserted {cursor.rowcount} rows.")
        else: print("    Warning: No data to insert.")
        cnxn.commit(); print(f"  Changes for '{final_table_name}' committed.")
    except Exception as e: print(f"  ERROR processing table '{final_table_name}': {e}"); cnxn.rollback()
    finally:
        if cursor: cursor.close()
        if cnxn: cnxn.close()
        print("  Connection closed.")
print(f"--- Step 3E: Load Table '{final_table_name}' --- END ---")


--- Step 3E: Load Table 'Logs' --- START ---
Processing 'Logs' with 2014 rows.
  Connection established.
  Creating table 'Logs'...
  Table 'Logs' created.
  Preparing and inserting rows...
  SUCCESS: Inserted -1 rows.
  Changes for 'Logs' committed.
  Connection closed.
--- Step 3E: Load Table 'Logs' --- END ---


In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import urllib
import datetime

print("\n--- Step 3G: Load Table 'ScheduledPayments' --- START ---")
logical_name = 'scheduled_payments'; final_table_name = 'ScheduledPayments'; pk_col = 'id'

if 'cleaned_dataframes' not in locals() or cleaned_dataframes.get(logical_name) is None or cleaned_dataframes.get(logical_name).empty:
    print(f"ERROR/SKIP: cleaned_dataframes['{logical_name}'] not found or empty.")
else:
    df_cleaned = cleaned_dataframes[logical_name]
    print(f"Processing '{final_table_name}' with {len(df_cleaned)} rows.")
    DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'; DB_DATABASE = 'BankingDB'; USE_TRUSTED_CONNECTION = True; driver = '{ODBC Driver 17 for SQL Server}'
    cnxn = None; cursor = None
    def get_sql_type(dtype):
        if pd.api.types.is_integer_dtype(dtype): return 'BIGINT'
        elif pd.api.types.is_float_dtype(dtype): return 'FLOAT'
        elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME2'
        elif pd.api.types.is_bool_dtype(dtype): return 'BIT'
        else: return 'NVARCHAR(MAX)'
    def convert_value(item):
        if pd.isna(item): return None
        if isinstance(item, (np.integer)): return int(item)
        if isinstance(item, (np.floating)): return float(item)
        if isinstance(item, (np.bool_)): return bool(item)
        return item
    try:
        conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};DATABASE={DB_DATABASE};Trusted_Connection=yes;")
        cnxn = pyodbc.connect(conn_str, autocommit=False)
        cursor = cnxn.cursor()
        print("  Connection established.")
        # CREATE TABLE
        print(f"  Creating table '{final_table_name}'...")
        column_defs, valid_sql_cols = [], []
        for col_name in df_cleaned.columns:
            if not isinstance(col_name, str) or not col_name: continue
            sql_col_name=col_name; sql_type=get_sql_type(df_cleaned[col_name].dtype)
            column_defs.append(f"[{sql_col_name}] {sql_type}"); valid_sql_cols.append(sql_col_name)
        pk_def = f", CONSTRAINT PK_{final_table_name} PRIMARY KEY ([{pk_col}])" if pk_col in valid_sql_cols else ""
        create_sql = f"CREATE TABLE [{final_table_name}] ({', '.join(column_defs)} {pk_def} )"
        cursor.execute(f"DROP TABLE IF EXISTS [{final_table_name}]")
        cursor.execute(create_sql); print(f"  Table '{final_table_name}' created.")
        # INSERT INTO
        print(f"  Preparing and inserting rows...")
        sql_cols_str = ', '.join([f'[{c}]' for c in valid_sql_cols])
        placeholders = ', '.join('?' * len(valid_sql_cols))
        sql_insert = f"INSERT INTO [{final_table_name}] ({sql_cols_str}) VALUES ({placeholders})"
        data_tuples = [tuple(convert_value(item) for item in row) for row in df_cleaned[valid_sql_cols].itertuples(index=False, name=None)]
        if data_tuples:
            cursor.fast_executemany = True; cursor.executemany(sql_insert, data_tuples)
            print(f"  SUCCESS: Inserted {cursor.rowcount} rows.")
        else: print("    Warning: No data to insert.")
        cnxn.commit(); print(f"  Changes for '{final_table_name}' committed.")
    except Exception as e: print(f"  ERROR processing table '{final_table_name}': {e}"); cnxn.rollback()
    finally:
        if cursor: cursor.close()
        if cnxn: cnxn.close()
        print("  Connection closed.")
print(f"--- Step 3G: Load Table '{final_table_name}' --- END ---")


--- Step 3G: Load Table 'ScheduledPayments' --- START ---
Processing 'ScheduledPayments' with 800 rows.
  Connection established.
  Creating table 'ScheduledPayments'...
  Table 'ScheduledPayments' created.
  Preparing and inserting rows...
  SUCCESS: Inserted -1 rows.
  Changes for 'ScheduledPayments' committed.
  Connection closed.
--- Step 3G: Load Table 'ScheduledPayments' --- END ---


In [ ]:
import pyodbc
import urllib

print("\n--- Step 3H (Final FK Fix): Add Foreign Keys --- START ---")

DB_SERVER = 'WIN-A5I5TM5OKQQ\\SQLEXPRESS'; DB_DATABASE = 'BankingDB'; USE_TRUSTED_CONNECTION = True; driver = '{ODBC Driver 17 for SQL Server}'

scheduled_payments_card_column = 'card_id' 

foreign_keys_to_create = [
    ('Cards', 'FK_Cards_Users', 'user_id', 'Users', 'id'),
    ('Transactions', 'FK_Transactions_Cards_From', 'from_card_id', 'Cards', 'id'),
    ('Transactions', 'FK_Transactions_Cards_To', 'to_card_id', 'Cards', 'id'),
    ('Logs', 'FK_Logs_Transactions', 'transaction_id', 'Transactions', 'id'),
    # ScheduledPayments uchun child_column endi yuqoridagi o'zgaruvchidan olinadi
    ('ScheduledPayments', 'FK_ScheduledPayments_Cards', scheduled_payments_card_column, 'Cards', 'id')
]

# --- Tekshirish: Placeholder o'zgartirilganmi? ---
if 'HAQIQIY_USTUN_NOMINI_BU_YERGA_YOZING' in scheduled_payments_card_column:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("XATO: 'scheduled_payments_card_column' o'zgaruvchisiga haqiqiy ustun nomini kiritmadingiz!")
    print("Iltimos, SSMSdan 'dbo.ScheduledPayments' jadvalidagi kartaga ishora qiluvchi ustun nomini toping va koddagi 'HAQIQIY_USTUN_NOMINI_BU_YERGA_YOZING' o'rniga yozing.")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    exit() # Skriptni to'xtatish

cnxn = None; cursor = None
print("\n[FK Add] Setting up connection...")
try:
    conn_str = (f"DRIVER={driver};SERVER={DB_SERVER};DATABASE={DB_DATABASE};Trusted_Connection=yes;")
    cnxn = pyodbc.connect(conn_str, autocommit=False) # Autocommit FALSE
    cursor = cnxn.cursor()
    print("[FK Add] Connection established.")

    # --- FKlarni Qo'shish ---
    print("[FK Add] Attempting to add Foreign Keys...")
    all_fks_added = True
    # Avval o'chirish
    print("  Attempting to drop existing FKs first...")
    for child_table, constraint_name, _, _, _ in reversed(foreign_keys_to_create):
        try:
            cursor.execute(f"IF OBJECT_ID(?, 'F') IS NOT NULL ALTER TABLE [{child_table}] DROP CONSTRAINT [{constraint_name}]", (constraint_name,))
        except Exception: pass
    print("  Finished attempting to drop existing FKs.")
    cnxn.commit()

    # Endi qo'shish
    print("  Attempting to CREATE Foreign Keys...")
    for child_table, constraint_name, child_column, parent_table, parent_column in foreign_keys_to_create:
        print(f"    Adding FK: {constraint_name} ({child_table}.{child_column} -> {parent_table}.{parent_column})...")
        try:
            sql_fk = f"""
                ALTER TABLE [{child_table}] ADD CONSTRAINT [{constraint_name}]
                FOREIGN KEY ([{child_column}]) REFERENCES [{parent_table}]([{parent_column}])
            """
            cursor.execute(sql_fk)
            print(f"      FK '{constraint_name}' add command prepared.")
        except pyodbc.Error as e:
            print(f"      ERROR preparing/adding Foreign Key '{constraint_name}': {e}")
            all_fks_added = False
        except Exception as e:
            print(f"      Unexpected ERROR adding FK '{constraint_name}': {e}")
            all_fks_added = False

    # --- Yakuniy Commit yoki Rollback ---
    if all_fks_added:
        print("\n[FK Add] Committing FK changes...")
        cnxn.commit()
        print("[FK Add] FK changes committed successfully.")
    else:
        print("\n[FK Add] Errors occurred during FK creation. Rolling back...")
        cnxn.rollback()
        print("[FK Add] Rollback complete due to FK errors.")

except Exception as e:
    print(f"\n[FK Add] A MAJOR error occurred: {e}")
    if cnxn: cnxn.rollback()
finally:
    print("\n[FK Add] Closing connection...")
    if cursor: cursor.close()
    if cnxn: cnxn.close()
    print("[FK Add] Connection closed.")

print("\n--- Step 3H (Final FK Fix): Add Foreign Keys --- END ---")


--- Step 3H (Final FK Fix): Add Foreign Keys --- START ---

[FK Add] Setting up connection...
[FK Add] Connection established.
[FK Add] Attempting to add Foreign Keys...
  Attempting to drop existing FKs first...
  Finished attempting to drop existing FKs.
  Attempting to CREATE Foreign Keys...
    Adding FK: FK_Cards_Users (Cards.user_id -> Users.id)...
      FK 'FK_Cards_Users' add command prepared.
    Adding FK: FK_Transactions_Cards_From (Transactions.from_card_id -> Cards.id)...
      FK 'FK_Transactions_Cards_From' add command prepared.
    Adding FK: FK_Transactions_Cards_To (Transactions.to_card_id -> Cards.id)...
      FK 'FK_Transactions_Cards_To' add command prepared.
    Adding FK: FK_Logs_Transactions (Logs.transaction_id -> Transactions.id)...
      FK 'FK_Logs_Transactions' add command prepared.
    Adding FK: FK_ScheduledPayments_Cards (ScheduledPayments.card_id -> Cards.id)...
      FK 'FK_ScheduledPayments_Cards' add command prepared.

[FK Add] Committing FK changes